In [ ]:
#เดะลองรันwith L4 Gpu

In [ ]:
import time 
import pandas as pd 
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

def ReturnJsonFormat(Text_Json_format):
    Text_Json = "{" + Text_Json_format + "}"
    Text_Json = Text_Json.replace("'",'"').replace('[','}').replace(']','}')
    # print(Text_Json)
    return json.loads(Text_Json)


model_path="openthaigpt/openthaigpt-1.0.0-70b-chat"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, torch_dtype=torch.float16)
model.to(device)
# print(model.par)


def QueryLLM(Prompt,device='cuda',JsonFormat=False):
    inputs = tokenizer.encode(Prompt, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model.generate(inputs, max_length=512, num_return_sequences=1)
    Answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if JsonFormat:
        Answer = ReturnJsonFormat(Answer)
    return Answer

## Prompt Number 1 Content

In [ ]:
#Create content 
Topic = "create a chat gpt"
Prompt_Task1 = f"""You are a planner teacher for student learning. The Student want to learn how to {Topic}. So Create a guideline that he should study skill or knowledge before {Topic} 
write the Answer in the Json format and English Language 

'Learning1' : ['Topic' :'q1',
                    'detail' : 'd1'
                ],
                    
'Learning2' : 'Topic' :'q2',
                'detail' : 'd2'
                ]
"""
print(Prompt_Task1)

#### Return date frame

In [ ]:
Content_json_task1 = QueryLLM(Prompt_Task1,JsonFormat=True)
df = pd.DataFrame([
    {"Topic": q_info['Topic'],"detail": q_info['detail']}
    for q_key, q_info in Content_json_task1.items()
])
df 

## Prompt Number 2 10 Question

In [ ]:
#generate 10 question 
Topic = "create 10 question about chemistry"

Prompt_Task2 = f"""you are chem teacher who create exam for secondary student  {Topic}. So Create a guideline what should he learning for {Topic} so create chemistry quiz about org chem for secondary student
write the Answer in the Json format and English Language 

'Question1' : ['Question' :'q1',
                    'Choice' : [
                        'Choice1' : 'q1',
                        'Choice2' : 'q1',
                        'Choice3' : 'q1',
                        'Choice4' : 'q1'
                    ]],
                    
    'Question2' : ['Question' :'q2',
                    'Choice' : [
                        'Choice1' : 'q2',
                        'Choice2' : 'q2',
                        'Choice3' : 'q2',
                        'Choice4' : 'q2'
                    ]]
"""
print(Prompt_Task2)

Return Result w/ Dataframe

In [ ]:
Question_Json_Format = QueryLLM(Prompt_Task2,JsonFormat=True)
df = pd.DataFrame([
    {"Question": q_info['Question'], **q_info['Choice']}
    for q_key, q_info in Question_Json_Format.items()
])
df

## Prompt Number 3 Possible?

In [ ]:
#Possible / Impossible และ มีเส้นทางให้userเลือก
Persona = "greatest planner teacher for student learning"
Topic = "create chatGPT" #=====================================
Time = "24 hour"

Prompt_Task3 = f"""You are a {Persona}. The student want to learn to {Topic} in {Time}.
So tell me this student is possible to  learn this skill in {Time}.
Answer In Text Format. Like 'Possible' or 'Impossible' 
 When answer is 'impossible' add the reason why and answer in format
 Reason : [Reason] 
 When answer is 'impossible' make a choice to let's student choose another path to achieve the goal in format.

 'or you can choose another path'
 Path1 : [Path1]
 Path2 : [Path2]
 Path3 : [Path3]
 Path4 : [Path4]
 ...
"""



print(Prompt_Task3)

In [ ]:
Possbile_Json_Format = QueryLLM(Prompt_Task3)
Possbile_Json_Format